<a href="https://colab.research.google.com/github/thomasgiannetti/dotfiles/blob/master/your_first_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning - Day 4 - Exercise 1

You will here build your first Recurrent Neural Network (RNN). Not to consider to complex data, we will here just make the model compile and run on random data, to see the effect of the different options and parameters. So no need to interpret the results here.


# The data

We will create fake/random data here. 

❓ **Question** ❓ Create fake data where :
- Each observation is of size 3: `[2.2, 3.8, -1.2]` for instance.
- The first sequence `X_1` has 4 observations. 
- The first sequence `X_2` has 7 observations. 
- The third sequence `X_3` has 1 observation. 
- The last sequence `X_4` has 3 observations.
- Each output `y_1` to`y_4` is a scalar value - let's say that it will be a regression task with one value to predict

❗ **Remark** ❗ You can image that each observation corresponds to a job experience where the three features are (duration of the job, salary of the job, number of person under your responsability). And `X_1` thus corresponds to a sequence of 3 jobs, that someone has done. Same goes for `X_2` that is a sequence of 7 jobs, `X_3` and `X_4`. And the $y$ corresponds to the salary of the next job that you are trying to predict.

❗ **Remark** ❗ Make sure that some of the data are floats, not integers.

Hint: You can, for instance, use `numpy.random.normal`



At the end, the data should be store in $X$ and $y$ 

In [21]:
import numpy as np 
from numpy import random


X_1 = np.random.normal(0,10, size = (4,3))
X_2 = np.random.normal(0,10, size = (7,3))
X_3 = np.random.normal(0,10, size = (1,3))
X_4 = np.random.normal(0,10, size = (3,3))

y_1 = 1187
y_2 = 3160
y_3 = 3430
y_4 = 1270

X = np.array([X_1, X_2, X_3, X_4])
y = np.array([y_1, y_2, y_3, y_4])

You remember that we pass a batch of data to the neural network. Thus, the tensor will has the following shape (batch_size, number_of_observations, size_of_observation).

While size_of_observation is the same (=3) for all the sequences, the number_of_observations is different for each sequence. But it can't be like that in practice, for computational reason. So we will have (batch_size, max(number_of_observations), size_of_observation), so that each sequence has the same size.

We will do that we the `pad_sequences` function of keras. It basically adds 0 to fill in the blank.

❓ **Question** ❓ Use the `pad_sequences` function on X directly (without extra arguments here), store the result in `X_pad` and print it

In [23]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_pad = pad_sequences(X)
X_pad

array([[[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   7,  -9],
        [-15,  -4,   1],
        [ 14,   1, -14],
        [-26,   4,   7]],

       [[  5,  -5, -10],
        [ 15,   3, -24],
        [ 11,  13,   6],
        [ -6, -10,  -4],
        [ -5,  17,  -7],
        [  4, -11,  -7],
        [ -1,   2,   1]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [-11,  -4,  14]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [ -7,   6,  14],
        [-18,   3,   6],
        [-11,  16,  -7]]], dtype=int32)

❓ **Question** ❓ You probably see that the returned data are integers, instead of float. To change that, turn the `dtype` argument to `float32`. Moreover, add the `0` at the end of the sequence thanks to the `padding` argument. And finally, in the case where your fake data contains the `0` values, you cannot pad with `0`, so choose another value with the `value` keyword.

❗ **Remark** ❗ Store the new result in `X_pad` and print it to be sure 

[See full documentation here](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences)

In [25]:
X_pad = pad_sequences(X, dtype= 'float32')
X_pad

array([[[  0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ],
        [  0.02750805,   7.219553  ,  -9.384193  ],
        [-15.965339  ,  -4.1666775 ,   1.6476686 ],
        [ 14.740024  ,   1.0163573 , -14.4203825 ],
        [-26.276098  ,   4.367518  ,   7.2126975 ]],

       [[  5.9514465 ,  -5.794809  , -10.53681   ],
        [ 15.35759   ,   3.3547924 , -24.008545  ],
        [ 11.27538   ,  13.289081  ,   6.280439  ],
        [ -6.547479  , -10.354711  ,  -4.4429703 ],
        [ -5.4258075 ,  17.317484  ,  -7.742754  ],
        [  4.4599166 , -11.102633  ,  -7.68347   ],
        [ -1.4227264 ,   2.2310643 ,   1.5362968 ]],

       [[  0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ],
        

❓ **Question** ❓ Print the `X_pad` shape. Check that the assert is verified

In [28]:
print(X_pad.shape)

(4, 7, 3)


In [33]:
assert(X_pad.shape == (4, 7, 3))

# The model

Now, you will create your first Recurrent Neural Network.

❓ **Question** ❓ Write a model that has: 
- a first Masking layer whose role is computational only: as you have padded the input data, you have to 'un-pad' them so that the Network does not use the padded values.
- a second LSTM layer ; choose any `units` number but don't forget to choose the `tanh` activation function
- a last Dense layer specific to your task

❗ **Remark** ❗ If you have padded your input data with values different from 0, you should inform the Masking layer about that.

In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

model = Sequential()
model.add(layers.Masking(mask_value=0))
model.add(layers.LSTM(units=1, activation='tanh')) 
model.add(layers.Dense(1, activation='linear'))

❓ **Question** ❓ Compile your model. Remember to first use the `rmsprop` optimizer (instead of Adam).

In [35]:
model.compile(loss='mse', 
              optimizer='rmsprop')

❓ **Question** ❓ Run your model on your data. Do not use any validation set nor early stopping criterion as this are fake data here

In [36]:
model.fit(X_pad, y)

1/1 [==============================] - 0s 1ms/step - loss: 6192963.0000


❓ **Question** ❓ Now, build Neural Network whose RNN part is a `GRU`. Compile it and fit it then.

In [37]:
model = Sequential()
model.add(layers.Masking(mask_value=0))
model.add(layers.GRU(units=1, activation='tanh')) 
model.add(layers.Dense(1, activation='linear'))
model.compile(loss='mse', 
              optimizer='rmsprop')
model.fit(X_pad, y)

1/1 [==============================] - 0s 1ms/step - loss: 6190489.0000


# Well done!

## You now know how to run RNN on sequence data!